In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# # parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# # parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250km; Np = 50M 
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# # parcel=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
#LOAD VARIABLES
################################################################################
def load_vars(data_t):
    # print('MOISTURE VARIABLES'); ################################# MOISTURE VARIABLES
    rv=data_t['qv'].data;rl=data_t['qc'].data+data_t['qr'].data;
    ri=data_t['qi'].data
    
    # print('THERMODYNAMICS'); ################################# THERMODYNAMICS
    Rd=287.04
    Rv=461.5
    eps = Rd / Rv
    # Cpd=1005.7 #+-2.5
    # Cpv=1870 #+-25
    # Cpl=4190 #+-30
    # Lv0=2.501e6
    # def Lv(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
    #     Llv=Lv0+(Cpv-Cpl)*(T-273.15) #should it be Cpl. is Cl the same?***
    #     return Llv
    
    # print('POTENTIAL TEMPERATURE'); ################################# POTENTIAL TEMPERATURE
    th=data_t['th'].data

    return th,rv,rl,ri,eps

def make_theta_v(th,rv,rl,ri,eps):
    # print('VIRTUAL POTENTIAL TEMPERATURE'); ################################# VIRTUAL POTENTIAL TEMPERATURE
    th_v = (th*(1+rv/eps))/(1+rv+rl+ri)

    # print('TEMPERATURE'); ################################# TEMPERATURE
    # T=theta*(P/p0)**(Rd/Cpd)
    
    # print('VIRTUAL TEMPERATURE'); ################################# VIRTUAL TEMPERATURE
    # eps=Rd/Rv
    # rv=data['qv'].data
    # scalar=(1+rv/eps)/(1+rv+rl+ri)
    # Tv=T*scalar
    VARs={'th_v': th_v}
    return VARs

In [ ]:
def GetOutputName():
    if res=="1km":
        dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    elif res=='250m':
        dir2='/mnt/lustre/koa/scratch/air673/'
    out_file=dir2 + 'Variable_Calculation/OUTPUT/' + 'theta_v'+f'_{res}_{t_res}'+'.h5'
    return out_file
    
def initiate_array(VarNames):
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=GetOutputName()

    with h5py.File(out_file, 'a') as f:
        for var_name in VarNames:
            if var_name not in f:
                f.create_dataset(
                    var_name,
                    shape=(t_size, z_size, y_size, x_size),
                    maxshape=(None, z_size, y_size, x_size),
                    dtype='float64',
                    chunks=(1, z_size, y_size, x_size)
                )

def add_timestep_at_index(VARS, index):
    out_file=GetOutputName()
    
    with h5py.File(out_file, 'a') as f:
        for var_name, timestep_data in VARS.items():
            if var_name in f:
                f[var_name][index] = timestep_data
            else:
                raise KeyError(f"Dataset '{var_name}' does not exist in {out_file}")

In [ ]:
#RUNNING

In [ ]:
#MAKING ARRAY TO STORE theta_v
VarNames=['th_v']
initiate_array(VarNames)

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    data_t=data.isel(time=t)
    [th,rv,rl,ri,eps] = load_vars(data_t)
    VARs=make_theta_v(th,rv,rl,ri,eps)
    add_timestep_at_index(VARs, t)

In [ ]:
# #READING FINAL OUTPUT
# # dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# dir2='/mnt/lustre/koa/scratch/air673/'
# file_path=dir2 + 'Variable_Calculation/' + 'theta_v'+f'_{res}_{t_res}'+'.h5'
# with h5py.File(file_path, 'a') as f:
#     # Access the existing dataset 'MSE'
#     dataset = f['th_v'][a:b]

In [ ]:
# plt.contourf(dataset[50,0])
# plt.colorbar(label='K')
# plt.title("theta_v at t = 50, z = 30 m")